# Metropolis Algorithm - Hopefield Network

Ismael CM
19/12/2022

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [232]:
M = 16
N = 16
red = np.zeros((M,N), dtype=np.uint8) 

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=uint8)

In [90]:
def read_patern(path='Patern/patern.txt'):
	patern = np.loadtxt(path, delimiter=',', dtype=np.uint8)
	return patern

In [197]:
patern = read_patern(path='Patern/skull.txt')

In [230]:
def patern_topoint(patern):
	"""Convert a patern (matrix) to a list of points (x,y) where matrix[i][j] == 1
	
	x = j and y = -i + len(patern)

	Explenation:
		j selects the row (vertical movement) and i selects the column (horizontal movement)
		In a plot, x corresponds to the horizontal axis and y to the vertical axis

		Reversing i (i.e. -i) is necessary because the matrix is read from top to bottom. This is the same as reversing the y axis in a plot, so now everything is in the 4th quadrant.
		Adding len(patern) traslates everything to the 1st quadrant.
	"""
	x = []
	y = []
	for i in range(len(patern)): 			# i is the row
		for j in range(len(patern[i])): 	# j is the column
			if patern[i][j] == 1:
				x.append(j) 		    	
				y.append(-i + len(patern)) 
	return x,y